<a href="https://colab.research.google.com/github/dTenebrae/nlp/blob/main/lesson5/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## Загрузка данных и импорт библиотек

In [39]:
!pip install pyconll

In [40]:
import pyconll
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report
import pandas as pd

from collections import Counter

import warnings
warnings.simplefilter("ignore")

In [41]:
!mkdir datasets

mkdir: cannot create directory ‘datasets’: File exists


In [42]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-10-29 09:07:38--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   128MB/s    in 0.6s    

2021-10-29 09:07:40 (128 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-10-29 09:07:40--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [43]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

## Преобразуем данные

In [44]:
%%time
train_result = []
for sentence in full_train:
    sent_list = []
    for token in sentence:
        sent_list.append((token.form, token.upos))
    train_result.append(sent_list)

test_result = []
for sentence in full_test:
    sent_list = []
    for token in sentence:
        sent_list.append((token.form, token.upos))
    test_result.append(sent_list)

test_sent = []
for sentence in full_test:
    sent_list = []
    for token in sentence:
        sent_list.append(token.form)
    test_sent.append(sent_list)

CPU times: user 1.04 s, sys: 13 ms, total: 1.05 s
Wall time: 1.07 s


In [45]:
len(train_result), len(test_result)

(48814, 6584)

## Taggers

In [46]:
unigram_tagger = UnigramTagger(train_result)
unigram_tagger.evaluate(test_result)

0.8772537323492737

In [47]:
bigram_tagger = BigramTagger(train_result, backoff=unigram_tagger)
bigram_tagger.evaluate(test_result)

0.8829828463586425

In [48]:
trigram_tagger = TrigramTagger(train_result, backoff=bigram_tagger)
trigram_tagger.evaluate(test_result)

0.882081353418933

In [49]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = UnigramTagger(train_result)
tag = backoff_tagger(train_result,  
                     [BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_result) 

0.882081353418933

In [50]:
bigram_tagger.tag(test_sent[2])

[('Это', 'PRON'),
 ('связано', 'VERB'),
 ('с', 'ADP'),
 ('тем', 'PRON'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('работа', 'NOUN'),
 ('каких-то', 'DET'),
 ('инструкций', 'NOUN'),
 ('алгоритма', None),
 ('может', 'VERB'),
 ('быть', 'AUX'),
 ('зависима', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('инструкций', 'NOUN'),
 ('или', 'CCONJ'),
 ('результатов', 'NOUN'),
 ('их', 'PRON'),
 ('работы', 'NOUN'),
 ('.', 'PUNCT')]

In [51]:
train_tok = []
train_label = []
for sent in train_result:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in test_result:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [52]:
cnt = Counter(train_label)
cnt

Counter({'ADJ': 85589,
         'ADP': 81963,
         'ADV': 44101,
         'AUX': 7535,
         'CCONJ': 30432,
         'DET': 21968,
         'INTJ': 78,
         'NOUN': 214497,
         'NO_TAG': 1047,
         'NUM': 13746,
         'PART': 26638,
         'PRON': 38438,
         'PROPN': 32401,
         'PUNCT': 157989,
         'SCONJ': 16219,
         'SYM': 840,
         'VERB': 97670,
         'X': 375})

In [53]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

**Hashing vectorizer**

In [54]:
hvectorizer = HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100)

In [55]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [56]:
lr = LogisticRegression(random_state=0, max_iter=150)
lr.fit(X_train, train_enc_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
y_test_pred = lr.predict(X_test)
y_train_pred = lr.predict(X_train)

accuracy_score(test_enc_labels, y_test_pred)

0.6896673743807502

In [58]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred, digits=3):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred, digits=digits))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred, digits=digits))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [59]:
get_classification_report(train_enc_labels, y_train_pred, test_enc_labels, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0      0.565     0.448     0.500     85589
           1      0.772     0.904     0.833     81963
           2      0.515     0.232     0.320     44101
           3      0.827     0.974     0.894      7535
           4      0.859     0.969     0.911     30432
           5      0.595     0.569     0.582     21968
           6      0.000     0.000     0.000        78
           7      0.553     0.732     0.630    214497
           8      1.000     1.000     1.000      1047
           9      0.734     0.635     0.681     13746
          10      0.734     0.601     0.661     26638
          11      0.527     0.564     0.545     38438
          12      0.468     0.071     0.124     32401
          13      0.998     1.000     0.999    157989
          14      0.784     0.906     0.841     16219
          15      1.000     0.971     0.986       840
          16      0.623     0.514     0.563     97670
          17      0.

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [8]:
!pip install corus

     |████████████████████████████████| 83 kB 1.7 MB/s 


In [9]:
import corus

In [10]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-10-27 07:54:36--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.54MB/s    in 1.2s    

2021-10-27 07:54:38 (1.54 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [11]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [12]:
!rm collection5.zip

In [13]:
!ls

Collection5  datasets  sample_data


In [14]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)


Ne5Markup(
    id='1154',
    text='\r\n\r\n\r\nРеморенко после назначения Каганова остается замглавы Минобрнауки\r\n\r\n\r\nИгорь Реморенко пока остается работать в своей должности, сообщил РИА Новости источник в правительстве России.\r\nЗаместитель министра образования и науки РФ Игорь Реморенко пока остается работать в своей должности, несмотря на то, что на пост замглавы Минобрнауки в среду был назначен Вениамин Каганов, который, как предполагалось, сменит Реморенко, сообщил РИА Новости источник в правительстве России.\r\n\r\n"В настоящее время Реморенко остается работать в министерстве, хотя предполагалось, что Каганов сменит его на посту замминистра", — сказал собеседник агентства.\r\n\r\nВ начале прошлой недели в СМИ появилась информация о том, что Реморенко намерен уйти в отставку по собственному желанию. Источник РИА Новости в правительстве сообщил, что на его место, вероятно, будет назначен Каганов.',
    spans=[Ne5Span(
         index='T1',
         type='PER',
         star

процедуры обработки взять из вебинарного ноутбука

In [15]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 235 kB 9.1 MB/s 
     |████████████████████████████████| 2.9 MB 51.9 MB/s 
     |████████████████████████████████| 3.3 MB 75.5 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 929 kB 8.8 MB/s 
     |████████████████████████████████| 859 kB 49.0 MB/s 
     |████████████████████████████████| 654 kB 46.5 MB/s 
     |████████████████████████████████| 8.2 MB 10.2 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 4.3 MB/s 
     |████████████████████████████████| 43

In [16]:
#!python -m deeppavlov install squad_bert
#!python -m deeppavlov install ner_ontonotes

In [17]:
import deeppavlov
from deeppavlov import configs, build_model

In [18]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2021-10-27 07:55:50.202 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/datasets/entity_and_type_detection_rus.pickle to /root/.deeppavlov/models/entity_and_type_detection_rus.pickle
100%|██████████| 2.07M/2.07M [00:01<00:00, 2.06MB/s]
2021-10-27 07:55:53.463 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/models/ner_cq_rus.tar.gz to /root/.deeppavlov/models/ner_cq_rus.tar.gz
100%|██████████| 1.32G/1.32G [04:40<00:00, 4.70MB/s]
2021-10-27 08:00:35.606 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/models/ner_cq_rus.tar.gz archive into /root/.deeppavlov/models/ner_ent_and_type_rus
2021-10-27 0

ModuleNotFoundError: ignored